In [1]:
import  sys
sys.path.insert(0, "d:\python\diabolo")
import pandas as pd
import numpy as np
import matplotlib
import scipy
import platform
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix,average_precision_score

from sklearn.model_selection import RandomizedSearchCV
from matplotlib import pyplot

from sklearn.metrics import accuracy_score

import sklearn.metrics 
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from skopt import BayesSearchCV

import xgboost as xgb



c:\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\python\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:

print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)
print('scipy version is.......', scipy.__version__)

def my_drop(df, col):

    if col in df:
        df.drop([col], axis=1, inplace=True)
    return df

def assign_selection(W):
    if W <4:
        return 1
    else:
        return 0




Python version is........... 3.6.5
scikit-learn version is..... 0.19.1
pandas version is........... 0.22.0
numpy version is............ 1.14.2
matplotlib version is....... 2.2.0
scipy version is....... 1.0.0


# PROCEDURE DE BASES

In [3]:


def save_mymodel(clf,allure):
    joblib.dump(clf,'diabolo'+str(allure)+'.pkl')


def load_mymodel(allure):
    clf=joblib.load('diabolo'+str(allure)+'.pkl')
    return clf

class Timer:
    def __init__(self):
        self.start = time.time()

    def restart(self):
        self.start = time.time()

    def get_time(self):
        end = time.time()
        m, s = divmod(end - self.start, 60)
        h, m = divmod(m, 60)
        time_str = "%02d:%02d:%02d" % (h, m, s)
        return time_str






In [4]:
#Définir une méthode pour tracer l'importance du prédicteur
def Plot_predictor_importance(best_model, feature_columns):
    feature_importance = best_model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    y_pos  = np.arange(sorted_idx.shape[0]) + .7
    fig, ax = plt.subplots( figsize=(10, 10))



    ax.barh(y_pos,
            feature_importance[sorted_idx],
            align='center',
            color='green',
            ecolor='black',
            height=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_columns)
    ax.invert_yaxis()



    ax.set_xlabel('Relative Importance')
    ax.set_title('Predictor Importance')
    plt.show()


In [5]:
def construireFichierCSV(allure, best_model, feature_columns, response_column, mode_debug=0):
    # lecture des données a jouer


    xnames = ['ALLURE', 'CO_DISTANCE',
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE', 'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant', 'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'CORDE',
              'CHEVAL',
              'FIN_ligne']



    df_numero_a_predire = lecture_data.lecture_data('d:\diabollo_al_1_D.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure)



    if allure==5:
        print("\nSHAPE Numero a predire")
        print(df_numero_a_predire.shape)



    df_numero_a_predire = df_numero_a_predire.groupby("ALLURE")
    df_numero_a_predire = df_numero_a_predire.get_group(allure)
    df_numero_a_predire = lecture_data.my_drop(df_numero_a_predire, 'ALLURE')

    df_numero_a_predire = lecture_data.my_drop(df_numero_a_predire, 'PAR_ARRIVE')

    if mode_debug == 1:
        print("\nSHAPE Numero a predire")
        print(df_numero_a_predire.shape)

    my_timer = analyse.Timer()


    df_numero_a_predire['SELECTION'] = 0

    train_x, test_x, train_y, test_y = analyse.split_dataset(df_numero_a_predire, 0, feature_columns, response_column)
   # print(test_x.head(2))
    print(test_x.shape)


    # sauvegarde
    y_pred = best_model.predict(test_x)
    if mode_debug == 1:
        print("y pred")
        print(y_pred)

    df_pred = pd.DataFrame.from_dict(y_pred)
    test_copy = test_x.copy()



    # print(df_pred)
    #print("PREDICTION COURSES A JOUER")
    if mode_debug==1:
        print(y_pred)


    # UTILISATION DE BEST_MODEL
    PROBA = best_model.predict_proba(test_x)
    #print("PROBA \n %s" ,PROBA)

    df_proba = pd.DataFrame.from_dict(PROBA)

    #print("df final=")
    df_final = pd.concat([df_proba, df_pred], axis=1)

    if mode_debug==1:
        print("SHAPE df_final")
        print(df_final.shape)


    test_x = test_copy.copy()

    test_x = lecture_data.my_drop(test_x, 'PAR_CLASSE_AGE')

    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_ECART_GAGNANT')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_RAPPORT_GAGNANT_M')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_REU_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_GAIN')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_ECART_GAGNANT')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_ECART_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_NB_COURSE')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_PLACE_3P')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_QUINTE')
    test_x = lecture_data.my_drop(test_x, 'PAR_RUESSITE_PLACE')
    test_x = lecture_data.my_drop(test_x, 'CO_PRIX')
    test_x = lecture_data.my_drop(test_x, 'PAR_CARRIERE')
    test_x = lecture_data.my_drop(test_x, 'PAR_CARRIERE_Q')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_RAPPORT_GAGNANT_M')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_PLACE_Q')
    test_x = lecture_data.my_drop(test_x, 'PAR_CLASSE_AGE')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_3P')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_REU_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_NUM')
    test_x = lecture_data.my_drop(test_x, 'PAR_COTEDER')
    test_x = lecture_data.my_drop(test_x, 'CORDE')
    test_x = lecture_data.my_drop(test_x, 'musique')
    test_x = lecture_data.my_drop(test_x, 'CHEVAL')
    test_x = lecture_data.my_drop(test_x, 'Nb_partant')

    test_x = lecture_data.my_drop(test_x, 'autostart')
    test_x = lecture_data.my_drop(test_x, 'grande_piste')
    test_x = lecture_data.my_drop(test_x, 'cendre')

    test_x = lecture_data.my_drop(test_x, 'PAR_PROPRIO')
    test_x = lecture_data.my_drop(test_x, 'NOM_JOC')
    test_x = lecture_data.my_drop(test_x, 'NOM_ENTR')

    test_x = lecture_data.my_drop(test_x, 'HIPPO')
    test_x = lecture_data.my_drop(test_x, 'PAR_AGE')
    test_x = lecture_data.my_drop(test_x, 'POIDS')
    test_x = lecture_data.my_drop(test_x, 'CO_DISTANCE')
    test_x = lecture_data.my_drop(test_x, 'CO_PRIX')

    #print("test_x",test_x.shape)




    #print(test_x)


    test_x['v0'] = 0.0
    test_x['v1'] = 0.0
    test_x['sel'] = 0
    #   print("test_x", test_x.shape)
    # test_x
    cumul = 1
    nb_rows = len(df_proba.index)
    print("nb_rows=", nb_rows)

    for i in range(0, nb_rows):
        n = df_proba[0][i]
        test_x['v0'][i] = n

        n = df_proba[1][i]
        test_x['v1'][i] = n


    #print("test_x",test_x.shape)

   # print("\n\nTEST_X\n")
   # print(test_x.head())

    if (allure == 1):
        test_x.to_csv("d:\py_resultat_trot.csv")

    if (allure == 2):
        test_x.to_csv("d:\py_resultat_galop.csv")

    if (allure == 3):
        test_x.to_csv("d:\py_resultat_trot_monte.csv")

    if (allure == 4):
        test_x.to_csv("d:\py_resultat_haie.csv")

    if (allure == 5):
        test_x.to_csv("d:\py_resultat_steeple.csv")

    elapsed = my_timer.get_time()
    print("\nTemps de calcul de la construction du fichier  est : %s" % elapsed)
    print("\n\n ALLER DANS APPLICATION DIABOLO ....")



In [6]:


def lecture_data(Fichier, xnames, xindex_col, allure=1, mode_debug=0,avec_index=True):

    

    if  avec_index==True:
        df = pd.read_csv(Fichier,
                     index_col=xindex_col,
                     sep=';',
                     names=xnames, skipinitialspace=True,
                     encoding='Latin-1')
    else:
        df = pd.read_csv(Fichier,
                         index_col=None,
                         sep=';',
                         names=xnames, skipinitialspace=True,
                         encoding='Latin-1')



    print("\nDF Après LECTURE -------------- ",Fichier)

    print("\n",df.shape)
#    print("\n",df.head(5))


    if  avec_index==False:
        df.drop(["IDPARTCIPANT"], axis=1, inplace=True)
        df.drop(["IDCOURSE"], axis=1, inplace=True)


    df.drop(["FIN_ligne"], axis=1, inplace=True)
    df.drop(["PAR_NP"], axis=1, inplace=True)
    df.drop(["cendre"], axis=1, inplace=True)
   # df.drop(["CO_PRIX"], axis=1, inplace=True)


    df.drop(["PAR_CLASSE_AGE"], axis=1, inplace=True)

    #df.drop(["NOM_JOC"], axis=1, inplace=True)
    #df.drop(["NOM_ENTR"], axis=1, inplace=True)
    #df.drop(["CHEVAL"], axis=1, inplace=True)
    #df.drop(["PAR_PROPRIO"], axis=1, inplace=True)


   #df.drop(["PAR_AGE"], axis=1, inplace=True)
    df.drop(["grande_piste"], axis=1, inplace=True)
    #df.drop(["PAR_COTEDER"], axis=1, inplace=True)
    #df=my_drop(df, "HIPPO")
    df=my_drop(df, "PAR_AGE")
    df = my_drop(df, "PAR_GAIN")



    if allure==1:
        df.drop(["POIDS"], axis=1, inplace=True)
        df.drop(["CORDE"], axis=1, inplace=True)


    if allure == 2:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)

    if allure == 3:
        df.drop(["POIDS"], axis=1, inplace=True)
        df.drop(["CORDE"], axis=1, inplace=True)
        df.drop(["autostart"], axis=1, inplace=True)

    if allure == 4:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)
        df.drop(["CORDE"], axis=1, inplace=True)

    if allure == 5:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)
        df.drop(["CORDE"], axis=1, inplace=True)


    return df


def calul_data_allure(allure_etudier, mode_debug=0,avec_index=True):

    xnames = ['ALLURE'  , 'CO_DISTANCE',
              'CO_PRIX',  'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE',      'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant','PAR_PROPRIO',
                                         'NOM_JOC',
                                         'NOM_ENTR',
                                        'POIDS',
                                        'CORDE', 'CHEVAL',
                                         'FIN_ligne']








    df = lecture_data('d:\diabollo_al_1.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure_etudier,avec_index=avec_index)



    # groupage par allure
    df = df.groupby("ALLURE")
    df = df.get_group(allure_etudier)
    #df.head(5)
    df = my_drop(df, 'ALLURE')
    # Contruction de df_gagant
    df_gagnant = df
    # les lignes sans arrivée n'apportent rien !!!!
    df_gagnant = df_gagnant[df_gagnant.PAR_ARRIVE > 0]  # On garde la ligne qui possede information arrivée
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN < 20000000]
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN > 0]



    if allure_etudier==2:
        df_gagnant = df_gagnant[df_gagnant.POIDS > 0]
        df_gagnant = df_gagnant[df_gagnant.POIDS < 100]




    # Shape des données df_gagant
    if  mode_debug==1:
        print(" df_gagnant ", df_gagnant.shape)
        COLUM = df_gagnant.columns
        print("Features", COLUM )

    # Construction de la cible
    df_gagnant['SELECTION'] = df_gagnant['PAR_ARRIVE'].apply(assign_selection)

    #df_gagnant = df_gagnant[df_gagnant.PAR_COTEDER < 150]
     # Suppression de la colonne qui a permis de construire la cible : PAR_ARRIVE
    df_gagnant.drop(["PAR_ARRIVE"], axis=1, inplace=True)

    #fig, ax = plt.subplots(figsize=(16, 16))

    #sns.heatmap(df_gagnant.corr(), annot=True, fmt=".1f", linewidths=.4, ax=ax)
    #plt.show(fig)





#    print("\ndf_gagnant \n", df_gagnant.shape)  # description de l'ensemble
 #   pd.DataFrame.hist(df_gagnant, figsize = [18,18])



    df_gagnant_len            = len(df_gagnant.columns) - 1
    Lib_features                = list(df_gagnant.columns[:df_gagnant_len])  # Liebllé des variable DATA
    feature_columns         = Lib_features ##<<<<<<<<<<<<<<<<
    n_features = len(feature_columns) ##<<<<<<<<<<<<<<<<
    response_column = ['SELECTION']  ##<<<<<<<<<<<<<<<<

    print(df_gagnant.shape)  # lignes = observations
    print(df_gagnant.dtypes)  # lignes = observations


    print("Features                   :  ", Lib_features, " <<<<************")

    print("Nombre de feature   :  ", n_features)
    print("Colonne cible            : ", response_column)
    print("FEATURES COLONNES            : ", feature_columns)
    print("\n\n------------------------------------------------")




    # ---------------------------------------------------
    return df_gagnant, n_features, feature_columns, response_column




def donnees_allure(allure, nb_iter,mode_debug=0, actualise=0):
    # LECTURE DATA
    print('---------------------ALLURE(  %s  )------------------------------------------------' % allure)
    # Le fichier est lu meme si on  a le best model
    df_gagnant, n_features, feature_columns, response_column =calul_data_allure(allure_etudier = allure,mode_debug=mode_debug)

    return df_gagnant, n_features, feature_columns, response_colum



def save_mymodel(clf,allure):
    joblib.dump(clf,'diabolo'+str(allure)+'.pkl')


def load_mymodel(allure):
    clf=joblib.load('diabolo'+str(allure)+'.pkl')
    return clf






In [7]:

def afficheEvalSet(mybest_model,
                    set_train,
                   set_train_cible,
                   set_test,
                   set_test_cible,
                   mode_debug=0,
                   type_eval=1):


    set_test_cible_predicted = mybest_model.predict(set_test)
    set_train_cible_predicted = mybest_model.predict(set_train)

    print('Accuracy is: ', mybest_model.score(set_test, set_test_cible))  # accuracy


    if type_eval==1:
                print("Evaluation TEST-----------------------------------------------------------------------")

                set_test_cible_probabilities = mybest_model.predict_proba(set_test)
                auc = roc_auc_score(set_test_cible, set_test_cible_predicted)



    else:
                print("Evaluation TRAINING--------------------------------------------------------------------")

                set_train_cible_probabilities = mybest_model.predict_proba(set_train)
                auc = roc_auc_score(set_train_cible, set_train_cible_predicted)



    print("\n roc_auc_score :  ", auc)
    
    for x, y in [(set_train, set_train_cible), (set_test, set_test_cible)]:
            yp = mybest_model.predict(x)
            cm = confusion_matrix(y, yp.ravel())
            print(cm)
            

    import matplotlib.pyplot as plt

    
    plt.matshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    if type_eval==1:

        ntotal = len(set_test)
        correct = set_test_cible["SELECTION"].ravel() == set_test_cible_predicted
        numCorrect = sum(correct)
        percent = round((100.0 * numCorrect) / ntotal, 6)

        print("Classification Correcte des données de test : {0:d}/{1:d}  {2:8.3f}%".format(numCorrect, ntotal, percent))
        prediction_score = 100.0 * mybest_model.score(set_test, set_test_cible)
        print('\nScore  TEST  : %8.3f  ************************' % prediction_score)

    else:

        ntotal = len(set_train)
        correct = set_train_cible["SELECTION"].ravel() == set_train_cible_predicted
        numCorrect = sum(correct)
        percent = round((100.0 * numCorrect) / ntotal, 6)

        print("\n CLASSIFICATION CORRECTE DES DONNEES DE Train : {0:d}/{1:d}  {2:8.3f}%".format(numCorrect, ntotal, percent))
        prediction_score = 100.0 * mybest_model.score(set_train, set_train_cible)
        print('\nScore  TRAINING  : %8.3f  ************************' % prediction_score)
        
        


In [8]:
def modelfit(alg, dtrain,dtrain_y, predictors,target,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain_y[target].values, silent=False)
        
        print("xgb.cv ....................")
        
        cvresult = xgb.cv(xgb_param, 
                            xgtrain,
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds,
                          verbose_eval=True,
                          show_stdv=True,
                         metrics='auc', 
                          early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    print("Fit the algorithm on the data..............")
    alg.fit(dtrain[predictors], dtrain_y['SELECTION'],eval_metric='auc')
        
    print("Predict training set:")
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])
    
    
    
    
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(dtrain_y['SELECTION'].values, dtrain_predictions))
           

    
        
    return alg,cvresult,    dtrain_predictions,  dtrain_predprob

    

In [9]:
def entraine_allure_xgboost(allure, nb_iter, mode_debug=0, actualise=0):
    # LECTURE DATA
    global  df_gagnant, n_features, feature_columns, response_column , X_train, X_test, y_train, y_test
    
    

    
    print('---------------------ALLURE(  %s  )------------------------------------------------' % allure)
    # Le fichier est lu meme si on  a le best model
    df_gagnant, n_features, feature_columns, response_column = calul_data_allure(allure_etudier=allure, mode_debug=mode_debug)



    
    taille_training = 0.80
    

    X_train, X_test, y_train, y_test = train_test_split(df_gagnant[feature_columns], 
                                                        df_gagnant[response_column], 
                                                        test_size=1-taille_training,
                                                        random_state=42)
   
    
    


In [10]:
entraine_allure_xgboost(allure=1,   nb_iter=1,   mode_debug=0,   actualise=0)




---------------------ALLURE(  1  )------------------------------------------------

DF Après LECTURE --------------  d:\diabollo_al_1.csv

 (638048, 44)


c:\python\lib\site-packages\ipykernel_launcher.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python\lib\site-packages\ipykernel_launcher.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(325096, 34)
CO_DISTANCE                    int64
CO_PRIX                        int64
HIPPO                          int64
PAR_CARRIERE                   int64
PAR_CARRIERE_Q                 int64
PAR_COTEDER                    int64
PAR_ENT_ECART_GAGNANT          int64
PAR_ENT_RAPPORT_GAGNANT_M    float64
PAR_ENT_REU_PLACE            float64
PAR_ENT_REUSSITE_GAGNE       float64
PAR_ENT_VICTOIRE               int64
pAR_JOC_ECART_GAGNANT          int64
PAR_JOC_ECART_PLACE            int64
PAR_JOC_NB_COURSE              int64
PAR_JOC_PLACE_3P               int64
pAR_JOC_RAPPORT_GAGNANT_M    float64
PAR_JOC_REU_PLACE            float64
pAR_JOC_REUSSITE_GAGNE       float64
pAR_JOC_VICTOIRE               int64
PAR_NUM                        int64
PAR_PLACE                      int64
PAR_PLACE_Q                    int64
PAR_REUSSITE_3P              float64
PAR_REUSSITE_GAGNE           float64
PAR_REUSSITE_QUINTE          float64
PAR_RUESSITE_PLACE           float64
autostart                

In [11]:
# RECHERCHE DES BONS PARAMETRES

In [12]:
params={
    'learning_rate': [1 / i for i in range(10, 100)],
    'n_estimators':[75,100,125],
    'max_depth':[5,6,8,9],
    'min_child_weight': [3,5, 7, 9,11]
    
    
    

}

gsearch1 = RandomizedSearchCV(estimator = xgb.XGBClassifier( subsample=0.9,
                                                             colsample_bytree=0.8,
                                                             silent=False   ,
                                                       gamma=0,    objective= 'binary:logistic', 
                                                   scale_pos_weight=1,   seed=1000)  , 
                                                   n_iter =50,
                                                 param_distributions=params, 
                                        scoring='roc_auc', verbose=True,iid=False, cv=10  ,refit=True)          

gsearch1.fit(X_train,y_train['SELECTION'].ravel())



Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
print("\n",gsearch1.grid_scores_) 
print("\n",gsearch1.best_params_)
print("\n",gsearch1.best_score_)



In [ ]:
model = gsearch1.best_estimator_
eval_set=[(X_test, y_test)]
model.fit(X_train, y_train["SELECTION"].ravel(), eval_metric="error", eval_set=eval_set, verbose=False)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test["SELECTION"].ravel(), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(X_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=False)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()    
        

In [ ]:
#evaluation

In [ ]:
Plot_predictor_importance(model,feature_columns)